In [ ]:
import numpy as np
import pandas as pd
import os 

import tensorflow as tf
from sklearn.model_selection import train_test_split

from DeepModel import DeepModel
from Scaling import Scaling

%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.utils import shuffle

#tensorboard --logdir=logs/
#!{sys.executable} -m pip install tensorflow-addons 

In [ ]:
model_folder = '../model'

checkpoint_path = model_folder + "/TerrAI.ckpt"

In [ ]:
if not os.path.exists(model_folder):
    os.mkdir(model_folder)

In [ ]:
scaler = Scaling()

## Ensure available GPU utilized correctly

In [ ]:
physical_devices = tf.config.list_physical_devices("GPU")

if len(physical_devices) > 0:
    tf.config.experimental.set_memory_growth(physical_devices[0], True)

## Load training data

In [ ]:
data = pd.read_csv('../Dataset/Train.csv').dropna()
data.shape

In [ ]:
# Select main columns to be used in training
X = scaler.reshape_and_scale_X(data)
y = np.expand_dims(data.Label.to_numpy(dtype=np.float64), axis=1)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=2022)

In [ ]:
X.shape, y.shape

In [ ]:
input_size = X_train.shape[1:]
input_size

## Calculate the output class distribution percentage
Used in order to ensure class weight adjusted properly

In [ ]:
zero_state_count = len(y[y == 0])
total_label_count = len(y)
zero_state_count, total_label_count

In [ ]:
zero_weight = 1
one_weight = 1 / total_label_count * zero_state_count + 1

class_weight = {0: zero_weight, 1: one_weight}
class_weight

## Dataset generator

In [ ]:
batch_size = 512

In [ ]:
def fire(X, y, batch_size=64):
    return_index = 0
    max_index = len(y) // batch_size
    X, y = shuffle(X, y)
    
    for i in range(1, max_index):
        yield X[batch_size * (i-1):batch_size * i], y[batch_size * (i-1):batch_size * i]

In [ ]:
train_dataset = tf.data.Dataset.from_generator(
    generator=lambda: fire(X=X_train, y=y_train, batch_size=batch_size),
    output_types=(tf.float64, tf.float64),
    output_shapes=([None] + list(input_size), [None, 1]))

test_dataset = tf.data.Dataset.from_generator(
    generator=lambda: fire(X=X_test, y=y_test, batch_size=batch_size),
    output_types=(tf.float64, tf.float64),
    output_shapes=([None] + list(input_size), [None, 1]))

In [ ]:
fire_iter = iter(fire(X=X_test, y=y_test))
a, b = next(fire_iter)
a.shape, b.shape

In [ ]:
model = DeepModel(checkpoint_path, 1, input_size)

In [ ]:
model.train(train_dataset, test_dataset, class_weight=class_weight, epochs=250, batch_size=batch_size)